# NEVI

The NEVI project aims to prove usefullness of CNN for skin cancer recognition using image processing and deep convolutional neural networks with transfer learning.

By Severity

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.efficientnet_v2 import EfficientNetV2S, preprocess_input
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.layers import ThresholdedReLU
from keras.models import load_model
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import PIL


In [ ]:
SEVERITY_DATA_FOLDER_PATH = "grouped_images_by_severity"
BENIGN_DATA_FOLDER_PATH = "grouped_images_by_severity\\benign\\*"
MALIGNANT_DATA_FOLDER_PATH = "grouped_images_by_severity\\malignant\\*"
TARGET_SIZE = (256, 256)
INPUT_SHAPE = (256, 256, 3)
BATCH_SIZE = 20

In [ ]:
imgs_benign = glob.glob(BENIGN_DATA_FOLDER_PATH)
imgs_malignant = glob.glob(MALIGNANT_DATA_FOLDER_PATH)

df_benign = pd.DataFrame({'filename': imgs_benign})
df_benign['class'] = 'benign'
df_malignant = pd.DataFrame({'filename': imgs_malignant})
df_malignant['class'] = 'melignant'

df = pd.concat([df_benign, df_malignant])

df_train, df_temp = train_test_split(df, stratify=df['class'], test_size=.1)
df_val, df_test = train_test_split(df_temp, stratify=df_temp['class'], test_size=.5)


In [ ]:
data_generator_vis = ImageDataGenerator()
flow_vis = data_generator_vis.flow_from_directory(
    SEVERITY_DATA_FOLDER_PATH,
    shuffle=True
)

In [ ]:
data_generator_train = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=90,
    shear_range=.1,
    brightness_range=[0.6, 1.0]
)

data_generator_val = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

data_generator_test = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

In [ ]:
flow_train = data_generator_train.flow_from_dataframe(
    df_train,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

flow_val = data_generator_val.flow_from_dataframe(
    df_val,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
    
)

flow_test = data_generator_test.flow_from_dataframe(
    df_test,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

In [ ]:
EfficientNet = EfficientNetV2S(
    include_top=False,
    weights='imagenet',
    input_shape=INPUT_SHAPE,
    pooling='max'
)

In [ ]:
def get_model(base_model, activation="softmax", optimizer: str="adam", loss_function: str="CategoricalCrossentropy"):
    model = Sequential([
        base_model,
        Dropout(.2),
        Dense(2, activation=activation)
    ])
    
    for layer in model.layers[:-1]:
        layer.trainable=True
        
    model.compile(
        optimizer,
        loss_function,
        metrics=[
            "accuracy"
        ]
    )
    
    return model

In [ ]:
model_eff = get_model(EfficientNet)

In [ ]:
history_eff = model_eff.fit(
    flow_train,
    validation_data=flow_val,
    epochs=100
)

In [ ]:
_, eff_accuracy = model_eff.evaluate(flow_test)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
plt.plot(history_eff.history["accuracy"])
plt.plot(history_eff.history["val_accuracy"])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()

plt.figure(figsize=(8, 5))
plt.plot(history_eff.history["loss"])
plt.plot(history_eff.history["val_loss"])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()
